In [1]:
import os
import glob
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import ast
import string
import datetime
import re
from datetime import timedelta
pd.set_option("display.max_info_columns",200)
import psycopg2 as ps
from configparser import ConfigParser
#from sqlalchemy import create_engine
from dml_script import *

In [2]:
# Extract the configuration file
config = ConfigParser()
config.read("credentials.cfg")

['credentials.cfg']

In [3]:
#Creating the engine and setting up the database connections
rdbms = config.get("DB", "rdbms")
user = config.get("DB", "user")
password = config.get("DB", "password")
host = config.get("DB", "host")
port = config.get("DB", "port")
database = config.get("DB", "database")

In [4]:
#engine = create_engine(f'{rdbms}://{user}:{password}@{host}:{port}/{database}')
conn = ps.connect(f"host={host} dbname={database} user={user} password={password}")
cur = conn.cursor()
conn.set_session(autocommit=True)

In [2]:
df = pd.read_csv("c:/Users/vamsi/Desktop/proj_2.0/data.csv")
df
# df = df.dropna(axis=0,how='any',thresh=None,subset=None,inplace=False)
# df.info()

,InvoiceNo,Customer_ID,Date,Planet,Purchase
0,536365,17850.0,Dec 01 2010,Abydos,"[{'ItemNo': u'85123A', 'UnitPrice': 2.55, 'Des..."
1,536366,17850.0,40513,Abydos,"[{'ItemNo': 22633, 'UnitPrice': 1.85, 'Descrip..."
2,536367,13047.0,12/01/2010,Abydos,"[{'ItemNo': 84879, 'UnitPrice': 1.69, 'Descrip..."
3,536368,13047.0,Dec 01 2010,Abydos,"[{'ItemNo': 22960, 'UnitPrice': 4.25, 'Descrip..."
4,536369,13047.0,12/01/2010,Abydos,"[{'ItemNo': 21756, 'UnitPrice': 5.95, 'Descrip..."
5,536370,12583.0,40513,Altair IV,"[{'ItemNo': 22728, 'UnitPrice': 3.75, 'Descrip..."
6,536371,13748.0,2010-12-01 09:00:00,Abydos,"[{'ItemNo': 22086, 'UnitPrice': 2.55, 'Descrip..."
7,536372,17850.0,40513,Abydos,"[{'ItemNo': 22632, 'UnitPrice': 1.85, 'Descrip..."
8,536373,17850.0,Dec 01 2010,Abydos,"[{'ItemNo': u'85123A', 'UnitPrice': 2.55, 'Des..."
9,536374,15100.0,40513,Abydos,"[{'ItemNo': 21258, 'UnitPrice': 10.95, 'Descri..."


In [8]:
def update_date(date):
    date = date.strip()
    if re.match(r"^\d{8}$",date):
        date = datetime.datetime.strptime(date,'%Y%m%d')
    elif re.match(r"^\d{1,2}/", date):
        date = datetime.datetime.strptime(date,'%m/%d/%Y')
    elif re.match(r"^[a-z]{3}", date, re.IGNORECASE):
        date = datetime.datetime.strptime(date,'%b %d %Y')
    elif re.match(r"^\d{1,2} [a-z]{3}", date, re.IGNORECASE):
        date = datetime.datetime.strptime(date,'%d %b %Y')
    elif re.match(r"\b\d{5}\b", date) or re.match(r"\b\d{6}\b", date):
        date = (timedelta(days=int(date)) + datetime.datetime(1900,1,1)).strftime('%Y-%m-%d')
        '''    dateobj = datetime.strptime(dateobj,'%Y-%m-%d')
            print(dateobj)
            print('Date:',dateobj.date())'''
    #date = datetime.strptime(date,'%Y-%m-%d')
    return date

In [22]:
a = timedelta(minutes=243)
print(a)

4:03:00


In [14]:
df['Date'] = df.apply(lambda row: pd.Series(update_date(row['Date'])), axis=1)

In [11]:
t = df["Date"].drop_duplicates()
t = pd.to_datetime(t, unit = 'timestamp')
t.head()

0    2010-12-01 00:00:00
1    2010-12-03 00:00:00
6    2010-12-01 09:00:00
13   2010-12-01 09:37:00
14   2010-12-01 09:41:00
Name: Date, dtype: datetime64[ns]

In [12]:
time_data = ([t, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday])
column_labels = (['purchase_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'])

In [13]:
zip_data = zip(column_labels,time_data)
dict_data = dict(zip_data)
time_df = pd.DataFrame(dict_data).reset_index(drop=True)
time_df.head()

,purchase_time,hour,day,week,month,year,weekday
0,2010-12-01 00:00:00,0,1,48,12,2010,2
1,2010-12-03 00:00:00,0,3,48,12,2010,4
2,2010-12-01 09:00:00,9,1,48,12,2010,2
3,2010-12-01 09:37:00,9,1,48,12,2010,2
4,2010-12-01 09:41:00,9,1,48,12,2010,2


In [14]:
for i, row in time_df.iterrows():
    cur.execute(purchase_time_table_insert, list(row))
    #cur.execute("INSERT INTO time(start_time,hour,day,week,month,year,weekday)VALUES(%s,%s,%s,%s,%s,%s,%s)",
     #           list(row))
    #conn.commit()

In [7]:
#insert product records
df.rename(columns = {'Planet':'product_name'}, inplace = True)
df_product = df[['product_name']].drop_duplicates()
df_product

for i,row in df_product.iterrows():
    cur.execute(products_table_insert,row)
#conn.commit()
#df_product_name = df_product.product_name.unique()
#product_name = pd.DataFrame(df_product_name)
#product_name.values[0]

In [8]:
df_purchase = df['Purchase'].apply(ast.literal_eval)

In [9]:
df1 = pd.concat({k:pd.DataFrame(v) for k, v in df_purchase.items()})

In [10]:
df = df.join(df1.reset_index(level=1, drop=True)).reset_index(drop=True)

In [ ]:
'''df_product_item = df[['product_name','ItemNo','Description','Quantity','UnitPrice']].drop_duplicates()
#df_product_item
df_product_item.sort_values(by=['product_name','ItemNo'], inplace=True, ascending=True)
df_product_item.rename(columns = {'ItemNo':'item_code','UnitPrice':'unit_price'}, inplace = True)
df_product_item.info()'''

In [12]:
df1.rename(columns = {'ItemNo':'item_code','UnitPrice':'unit_price'}, inplace = True)
df_item = df1[['item_code','Description']].drop_duplicates()
#df_item1 = df_item[['item_code']].drop_duplicates()
#df_test = pd.merge(df_item1, df_item, on='item_code', how='left')
#df_test
#df_test = df_test.query("item_code== 22632");
#df_item
#df_item_no = pd.DataFrame(df_item_name)
#del df_item['item_code']
#df_test = df_item_1.join(df_item.reset_index(level=1, drop=True)).reset_index(drop=True)
#df_test
        
#df_item = df_item.query('item_code=22632')
#df_item.sort_values(by=[('item_code')])
#df_item.sort_values(by=['item_code'], inplace=True, ascending=True)

#df_item = [22632]
#df_item.sort_values(by=['22632'])

#insert item records
for i,row in df_item.iterrows():
    cur.execute(items_table_insert,row)

In [ ]:
#df_product_item = df1.drop_duplicates()
#d_test
#f_test = df_test.query("item_code== 22632");
#pd.DataFrame(df_product_item)

In [ ]:
'''for i,row in df_product_item.iterrows():
    cur.execute(product_select,(str(row.product_name),))
    v_product = cur.fetchone()
    if v_product is None:
        break
    else:
        cur.execute(item_select,(str(row.item_code),))
        v_item = cur.fetchone()
        if v_item is None:
            break
        else:
            product_item_detail_data = (v_product,v_item,row.unit_price,row.Quantity)
            cur.execute(product_item_details_table_insert,product_item_detail_data)'''

In [ ]:
#df_test = df_item_no.join(df_item.reset_index(level=1, drop=True)).reset_index(drop=True)
#df_test.drop_duplicates()

In [15]:
#insert item records
for i,row in df_item.iterrows():
    cur.execute(items_table_insert,row)
        #get product_id and item_id from products and items tables.

In [13]:
del df['Purchase']
#df.columns = ['InvoiceNo','Customer_ID','Date','Product','Product_Type','ItemNo','Quantity','UnitPrice']

In [15]:
df.rename(columns = {'ItemNo':'item_code','UnitPrice':'unit_price'}, inplace = True)
df

,InvoiceNo,Customer_ID,Date,product_name,Description,item_code,Quantity,unit_price
0,536365,17850.0,Dec 01 2010,Abydos,ADORIAN CROSSBOW,85123A,6,2.55
1,536365,17850.0,Dec 01 2010,Abydos,WHITE METAL LANTERN,71053,6,3.39
2,536365,17850.0,Dec 01 2010,Abydos,CREAM CUPID HEARTS COAT HANGER,84406B,8,2.75
3,536365,17850.0,Dec 01 2010,Abydos,KNITTED UNION FLAG HOT WATER BOTTLE,84029G,6,3.39
4,536365,17850.0,Dec 01 2010,Abydos,RED WOOLLY HOTTIE WHITE HEART.,84029E,6,3.39
5,536365,17850.0,Dec 01 2010,Abydos,SET 7 BABUSHKA NESTING BOXES,22752,2,7.65
6,536365,17850.0,Dec 01 2010,Abydos,GLASS STAR FROSTED T-LIGHT HOLDER,21730,6,4.25
7,536366,17850.0,40513,Abydos,BRAWNDO,22633,6,1.85
8,536366,17850.0,40513,Abydos,HAND WARMER RED POLKA DOT,22632,6,1.85
9,536367,13047.0,12/01/2010,Abydos,REVERBERATING CARBONIZER,84879,32,1.69


In [ ]:
'''df = df.loc[:,["Date"]]
df = df.head(10)
df['Date']'''

In [ ]:
#df_date = df['Date'].drop_duplicates()

In [ ]:
#df_date = pd.to_datetime(df_date, format='%Y-%m-%d').dt.strftime('%m/%d/%Y')

In [ ]:
#df_date = pd.DataFrame(df_date)

In [16]:
def update_date(date):
    date = date.strip()
    if re.match(r"^\d{8}$",date):
        date = datetime.datetime.strptime(date,'%Y%m%d')
    elif re.match(r"^\d{1,2}/", date):
        date = datetime.datetime.strptime(date,'%m/%d/%Y')
    elif re.match(r"^[a-z]{3}", date, re.IGNORECASE):
        date = datetime.datetime.strptime(date,'%b %d %Y')
    elif re.match(r"^\d{1,2} [a-z]{3}", date, re.IGNORECASE):
        date = datetime.datetime.strptime(date,'%d %b %Y')
    elif re.match(r"\b\d{5}\b", date) or re.match(r"\b\d{6}\b", date):
        date = (timedelta(days=int(date)) + datetime.datetime(1900,1,1)).strftime('%Y-%m-%d')
        '''    dateobj = datetime.strptime(dateobj,'%Y-%m-%d')
            print(dateobj)
            print('Date:',dateobj.date())'''
    #date = datetime.strptime(date,'%Y-%m-%d')
    return date

In [17]:
df['Date'] = df.apply(lambda row: pd.Series(update_date(row['Date'])), axis=1)
df

,InvoiceNo,Customer_ID,Date,product_name,Description,item_code,Quantity,unit_price
0,536365,17850.0,2010-12-01 00:00:00,Abydos,ADORIAN CROSSBOW,85123A,6,2.55
1,536365,17850.0,2010-12-01 00:00:00,Abydos,WHITE METAL LANTERN,71053,6,3.39
2,536365,17850.0,2010-12-01 00:00:00,Abydos,CREAM CUPID HEARTS COAT HANGER,84406B,8,2.75
3,536365,17850.0,2010-12-01 00:00:00,Abydos,KNITTED UNION FLAG HOT WATER BOTTLE,84029G,6,3.39
4,536365,17850.0,2010-12-01 00:00:00,Abydos,RED WOOLLY HOTTIE WHITE HEART.,84029E,6,3.39
5,536365,17850.0,2010-12-01 00:00:00,Abydos,SET 7 BABUSHKA NESTING BOXES,22752,2,7.65
6,536365,17850.0,2010-12-01 00:00:00,Abydos,GLASS STAR FROSTED T-LIGHT HOLDER,21730,6,4.25
7,536366,17850.0,2010-12-03 00:00:00,Abydos,BRAWNDO,22633,6,1.85
8,536366,17850.0,2010-12-03 00:00:00,Abydos,HAND WARMER RED POLKA DOT,22632,6,1.85
9,536367,13047.0,2010-12-01 00:00:00,Abydos,REVERBERATING CARBONIZER,84879,32,1.69


In [34]:
#df.info()
#df[['Customer_ID']] = df[['Customer_ID']].astype(int)
#df = df.query("customer_ID==' '");

#df_purchase.loc[df['Customer_ID'].notnull(), 'Customer_ID'] = df_purchase.loc[df['Customer_ID'].notnull(), 'Customer_ID'].apply(int)
#print(df_purchase[df_purchase['InvoiceNo'].isnull()])
#df_purchase.info()

In [20]:
df_purchase = df[['InvoiceNo','Customer_ID','Date','product_name','item_code','unit_price','Quantity']].drop_duplicates()
df_purchase.loc[df['Customer_ID'].notnull(), 'Customer_ID'] = df_purchase.loc[df['Customer_ID'].notnull(), 'Customer_ID'].apply(int)
#df_product_item
df_purchase.sort_values(by=['InvoiceNo','product_name','item_code'], inplace=True, ascending=True)
#df_purchase.rename(columns = {'ItemNo':'item_code','UnitPrice':'unit_price'}, inplace = True)
df_purchase

,InvoiceNo,Customer_ID,Date,product_name,item_code,unit_price,Quantity
6,536365,17850,2010-12-01 00:00:00,Abydos,21730,4.25,6
5,536365,17850,2010-12-01 00:00:00,Abydos,22752,7.65,2
1,536365,17850,2010-12-01 00:00:00,Abydos,71053,3.39,6
4,536365,17850,2010-12-01 00:00:00,Abydos,84029E,3.39,6
3,536365,17850,2010-12-01 00:00:00,Abydos,84029G,3.39,6
2,536365,17850,2010-12-01 00:00:00,Abydos,84406B,2.75,8
0,536365,17850,2010-12-01 00:00:00,Abydos,85123A,2.55,6
8,536366,17850,2010-12-03 00:00:00,Abydos,22632,1.85,6
7,536366,17850,2010-12-03 00:00:00,Abydos,22633,1.85,6
17,536367,13047,2010-12-01 00:00:00,Abydos,21754,5.95,3


In [21]:
for i,row in df_purchase.iterrows():
    cur.execute(product_select,(str(row.product_name),))
    v_product = cur.fetchone()
    if v_product is None:
        break
    else:
        cur.execute(item_select,(str(row.item_code),))
        v_item = cur.fetchone()
        if v_item is None:
            break
        else:
            purchase_data = (row.InvoiceNo,row.Date,row.Customer_ID,v_product,v_item,row.unit_price,row.Quantity)
            cur.execute(purchase_table_insert,purchase_data)

In [12]:
cur.close()